# Create and Install the requirements.txt

In [ ]:
%%writefile requirements.txt
colabcode
fastapi
uvicorn
python-multipart
ISR 

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
#An extra install due to some issues with the model as stated in the github issue of the model https://github.com/idealo/image-super-resolution/issues/197#issue-877826405
pip install 'h5py==2.10.0' --force-reinstall

#  Download and install Ngrok which is used for exposing the localhost so we can view the API with Google Colab.

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

--2022-06-10 18:34:01--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7247863 (6.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   6.91M  38.7MB/s    in 0.2s    

2022-06-10 18:34:02 (38.7 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [7247863/7247863]



In [ ]:
!tar -xvzf /content/ngrok-v3-stable-linux-amd64.tgz

ngrok


In [ ]:
!ngrok authtoken 29vr8YhWZ9CDHrUq2kr0CpUA0e8_6ik4hU5GjJZeAYagTH5i4

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Instantiate the Colab Code module and the run the next cell to create the fastapi app.

In [ ]:
#!pip install colabcode
from colabcode import ColabCode

In [ ]:
import uvicorn
from fastapi import FastAPI, File, UploadFile, Response
from fastapi.responses import StreamingResponse, FileResponse
import numpy as np
import io
from PIL import Image
from ISR.models import RDN
import cv2

app = FastAPI()

rdn = RDN(weights='noise-cancel')

@app.post("/")
async def root(file: UploadFile = File(...)):

    contents = io.BytesIO(await file.read())
    file_bytes = np.asarray(bytearray(contents.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

    sr_img = rdn.predict(img, by_patch_of_size=300)
    # Image.fromarray(sr_img)
    res, im_png = cv2.imencode(".png", sr_img)

    return StreamingResponse(io.BytesIO(im_png.tobytes()), media_type="image/png")



66076672/66071288 [==============================] - 3s 0us/step


In [ ]:
cc = ColabCode(port=18000, code=False)

# The API can then be viewed using the Public Url Generated by Colab Code.

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://8d4a-35-231-208-162.ngrok.io" -> "http://localhost:18000"


INFO:     Started server process [61]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:18000 (Press CTRL+C to quit)


INFO:     102.89.47.10:0 - "GET / HTTP/1.1" 405 Method Not Allowed
INFO:     102.89.47.10:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.89.47.10:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.89.47.10:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     102.89.46.179:0 - "POST / HTTP/1.1" 200 OK
